In [36]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
import config  # 引入配置文件
import csv
import time as time_module
from selenium.common.exceptions import TimeoutException
import json
import re
import requests


# 设置ChromeDriver的数据缓存目录
chrome_driver_dir = os.path.join(os.getcwd(), 'chromeDriver')
user_data_dir = os.path.join(chrome_driver_dir, 'UserData')

# 如果用户数据目录不存在，则创建它
if not os.path.exists(user_data_dir):
    os.makedirs(user_data_dir)

# 配置Chrome选项，以指定用户数据目录和其他启动参数
chrome_options = Options()
chrome_options.add_argument(f"--user-data-dir={user_data_dir}")  # 设置用户数据目录
chrome_options.add_argument("--no-first-run")  # 防止首次运行弹窗
chrome_options.add_argument("--no-default-browser-check")  # 防止检查默认浏览器
chrome_options.add_argument("--incognito")  # Chrome使用匿名模式
chrome_options.add_argument("window-size=1024,768")  # 设定Chrome窗口初始大小为1024*768
# 移除了无头模式的启动参数，以便打开浏览器窗口
# chrome_options.add_argument("--headless")  # 运行在无头模式

# 使用webdriver_manager来管理ChromeDriver
chrome_service = ChromeService(ChromeDriverManager().install())

# 初始化WebDriver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)


In [37]:
driver.get("https://cn.tradingview.com/")

In [ ]:
# 手工解决登录问题！
# 手工解决登录问题！
# 手工解决登录问题！

In [38]:
driver.get("https://cn.tradingview.com/ideas/followed-authors/page-2/")

# 手动点击“显示更多……”并翻页到最后再进行下一个单元格的数据处理！
# 手动点击“显示更多……”并翻页到最后再进行下一个单元格的数据处理！
# 手动点击“显示更多……”并翻页到最后再进行下一个单元格的数据处理！


In [46]:


# 寻找页面中所有荐股卡片
card_elements = driver.find_elements(By.CSS_SELECTOR, ".tv-widget-idea.js-userlink-popup-anchor")
# 初始化一个空数组来存储荐股信息
recommendation_info_list = []

# 遍历找到的荐股卡片元素
# 获取access_token
auth_url = "https://aip.baidubce.com/oauth/2.0/token"
auth_params = {
    "grant_type": "client_credentials",
    "client_id": "8qMmNgi5bdqMj7Z9tNrypYuK",  # 替换YOUR_API_KEY为实际的API Key
    "client_secret": "nMf2D6CwXui5WfxKTL0P55TS77LhVv5y"  # 替换YOUR_SECRET_KEY为实际的Secret Key
}
auth_response = requests.get(auth_url, params=auth_params)
if auth_response.status_code == 200:
    access_token = auth_response.json().get("access_token")
    print(f"Access Token: {access_token}")  # 打印获取的access_token
else:
    access_token = None
    print("获取access_token失败")

for card_element in card_elements:
    # 提取日期信息
    date_element = card_element.find_element(By.CSS_SELECTOR, ".tv-card-stats__time")
    title = date_element.get_attribute("title")
    data_timestamp = date_element.get_attribute("data-timestamp")
    
    # 提取证券名称、证券代码以及操作方向信息
    title_element = card_element.find_element(By.CSS_SELECTOR, ".tv-widget-idea__title")
    title_text = title_element.text
    parts = title_text.split(' ')
    if len(parts) == 3:  # 确保标题格式正确
        stock_name, stock_code, operation = parts
        # 只对stock_code为0或6开头的推荐卡片进行数据提取
        if not (stock_code.startswith('0') or stock_code.startswith('6')):
            continue
    else:
        continue  # 如果格式不正确，则跳过当前循环
    
    # 提取入场价格信息
    description_element = card_element.find_element(By.CSS_SELECTOR, ".tv-widget-idea__description-row")
    description_text = description_element.text
    
    if access_token:
        # 设置API请求的URL和参数
        api_url = f"https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions_pro?access_token={access_token}"
        modified_description_text = f"{description_text}\n" \
            "\n从这段话里找出“入场价”、\"止损位\"、\"第一目标位\"、\"第二目标位\"、\"第三目标位\"，回答模板：\n" \
            "\n" \
            "【入场价】：#,###.##（仅数字，不要任何文字说明）\n" \
            "【止损位】：#,###.##（仅数字，不要任何文字说明）\n" \
            "【第一目标位】：#,###.##（仅数字，不要任何文字说明）\n" \
            "【第二目标位】：#,###.##（仅数字，不要任何文字说明）\n" \
            "【第三目标位】：#,###.##（仅数字，不要任何文字说明）\n"
        print(f"Modified Description Text: {modified_description_text}")  # 打印修改后的描述文本
        params = {
            "messages": [
                {"role": "user", "content": modified_description_text}  # 将修改后的文本作为content的值
            ]
        }
        headers = {
            "Content-Type": "application/json"
        }
        
        # 发送POST请求到API
        response = requests.post(api_url, json=params, headers=headers)
        print(f"API Response: {response.json()}")  # 打印API响应
        if response.status_code == 200:
            response_data = response.json()
            # 从API返回的数据中提取格式为【入场价】：#,###.##的入场价格信息
            result_text = response_data.get("result", "未提供")
            print(f"Result Text: {result_text}")  # 打印结果文本
            
            # 使用正则表达式提取各项价格信息
            prices = re.findall(r"【\w+】：(\d+\.\d+|\d+)", result_text)
            # 初始化所有价格信息为None
            entry_price = stop_loss = first_target = second_target = third_target = None
            if prices:
                # 根据提取到的价格信息数量，分别赋值
                if len(prices) >= 1:
                    entry_price = prices[0]
                if len(prices) >= 2:
                    stop_loss = prices[1]
                if len(prices) >= 3:
                    first_target = prices[2]
                if len(prices) >= 4:
                    second_target = prices[3]
                if len(prices) >= 5:
                    third_target = prices[4]
                print(f"Entry Price: {entry_price}, Stop Loss: {stop_loss}, First Target: {first_target}, Second Target: {second_target}, Third Target: {third_target}")
            else:
                print("未匹配到价格信息")
        else:
            print("API请求失败，未提供价格信息")
    else:
        entry_price = stop_loss = first_target = second_target = third_target = "未获取access_token"
        print("未获取到access_token")
    # 将提取的信息存入字典
    recommendation_info = {
        "title": title,
        "data_timestamp": data_timestamp,
        "stock_name": stock_name,
        "stock_code": stock_code,
        "operation": operation,
        "entry_price": entry_price,
        "stop_loss": stop_loss,
        "first_target": first_target,
        "second_target": second_target,
        "third_target": third_target
    }
    print(f"Recommendation Info: {recommendation_info}")  # 打印推荐信息
    # 将字典添加到列表中
    recommendation_info_list.append(recommendation_info)
# 打印输出结果
print(json.dumps(recommendation_info_list, ensure_ascii=False, indent=2))



Access Token: 24.039a226d7c56df158933d9a6d2cc0587.2592000.1714962138.282335-59995871
Modified Description Text: 三一重工 600031 做多 看涨蝙蝠形态+机构订单原理+成交量分布VAL+K线二次确认=做多 交易计划A: 直接入场做多。（缺点：止损间距大，容易打止损） 入场位14.17，止损位13元。 第一目标位看16.50元附近，减半仓，推保护。 第二目标位看25.50元附近，再减半仓，推保护。 第三目标位看36.26元附近，再减半仓，推保护。 交易计划B： 等价格回调，小级别，分批入场。 目标位和交易计划A一样。 止损位放在11元下方。 优点：不容易打止损，盈亏比高，止损点位小。

从这段话里找出“入场价”、"止损位"、"第一目标位"、"第二目标位"、"第三目标位"，回答模板：

【入场价】：#,###.##（仅数字，不要任何文字说明）
【止损位】：#,###.##（仅数字，不要任何文字说明）
【第一目标位】：#,###.##（仅数字，不要任何文字说明）
【第二目标位】：#,###.##（仅数字，不要任何文字说明）
【第三目标位】：#,###.##（仅数字，不要任何文字说明）

API Response: {'id': 'as-5ti99k3awb', 'object': 'chat.completion', 'created': 1712370143, 'result': '【入场价】：14.17\n【止损位】：13\n【第一目标位】：16.50\n【第二目标位】：25.50\n【第三目标位】：36.26', 'is_truncated': False, 'need_clear_history': False, 'finish_reason': 'normal', 'usage': {'prompt_tokens': 289, 'completion_tokens': 49, 'total_tokens': 338}}
Result Text: 【入场价】：14.17
【止损位】：13
【第一目标位】：16.50
【第二目标位】：25.50
【第三目标位】：36.26
Entry Price: 14.17, Stop Loss: 13, First 